In [39]:
import pandas as pd
import numpy as np

# Get data

In [40]:
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# %%
europe_or_nearby = ['Austria', 'Belarus', 'Belgium', 'Bulgaria', 'Croatia', 'Czechia',
       'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
       'Hungary', 'Ireland', 'Italy', 'Luxembourg', 'Netherlands',
       'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
       'United Kingdom']

# %%
distance = pd.read_csv('data/controls/distance.csv', index_col=0)
subsidies = pd.read_csv('data/controls/energy_subsidies.csv', index_col=0)
urss = pd.read_csv('data/controls/ex_urss_influenced.csv', index_col=0)
gdppc = pd.read_csv('data/controls/GDPperCap.csv', index_col=0)
unemployment = pd.read_csv('data/controls/unemployment.csv', index_col=0)

# %%
# keep change in subsidies from 2021 to 2022
subsidies['increase'] = subsidies['2022'] - subsidies['2021']
subsidies = subsidies[['increase']].rename(columns={'increase': 'energy_subsidies_increase'})

# %%
# will talke 2022 (not the change), to account for country economic disparities

# keep only countries in Europe or nearby for gdppc
gdppc = gdppc.loc[gdppc.index.isin(europe_or_nearby)]
gdppc = gdppc[['2022']].rename(columns={'2022': 'gdppc'})

# %%
unemployment = unemployment.loc[unemployment.index.isin(europe_or_nearby)]
unemployment['increase'] = unemployment['2022'] - unemployment['2021']
unemployment = unemployment[['2022']].rename(columns={'2022': 'unemployment_increase'})

# %%
bartik = pd.read_csv('data/FINAL_BARTIK.csv', index_col=0)

# %%
approvals = pd.read_csv('../Design/data/Approvals_cleaned.csv')
approvals.drop(columns=['Sweden', 'Italy'], inplace=True)

def compute_approval_change(start_date, end_date):
    df = approvals.copy()
    df['Date'] = pd.to_datetime(df['Date'])  # ensure datetime format

    # Convert wide to long format
    df_long = df.melt(id_vars='Date', var_name='country', value_name='approval')

    # Filter date range
    df_long = df_long[(df_long['Date'] >= start_date) & (df_long['Date'] <= end_date)]

    # Calculate change from start to end per country
    start_approvals = df_long[df_long['Date'] == pd.to_datetime(start_date)]
    end_approvals = df_long[df_long['Date'] == pd.to_datetime(end_date)]

    merged = pd.merge(start_approvals, end_approvals, on='country', suffixes=('_start', '_end'))
    merged['approval_change'] = merged['approval_end'] - merged['approval_start']

    return merged[['country', 'approval_change']]

approval_change = compute_approval_change('2022-06-01', '2022-12-01').dropna()

# %%
approval_change.index = approval_change['country']
approval_change = approval_change[['approval_change']]

# %% [markdown]
# ### Merge all together

# %%
# merge all controls into one DataFrame
dataset = distance.join(subsidies).join(gdppc).join(unemployment).join(bartik).join(approval_change) # . join(urss)

# %%
dataset.dropna(inplace=True)

# %%
# normalize the controls exept for the approval change and ex_urss_influenced
for col in dataset.columns:
    if col not in ['approval_change']:
        dataset[col] = (dataset[col] - dataset[col].mean()) / dataset[col].std()

dataset


,distance_to_kyiv_km,energy_subsidies_increase,gdppc,unemployment_increase,bartik_iv,approval_change
country,,,,,,
Austria,-0.723659,-0.341913,0.146928,-0.156204,-0.408905,-3.0
Belgium,0.427343,-0.688853,0.105088,0.060580,-0.083559,-5.0
Bulgaria,-0.773786,1.277144,-1.030335,-0.427747,0.632631,-8.0
Croatia,-0.519050,0.005028,-0.883014,0.581913,-0.125074,1.0
Czechia,-0.593214,-0.804500,-0.583107,-1.194370,1.326173,-6.0
Denmark,-0.320450,-0.688853,0.633161,-0.365488,-0.852492,-3.0
Estonia,-0.710322,-0.226266,-0.577946,0.060955,1.461019,-2.0
Finland,-0.604646,-0.688853,0.093822,0.491524,-0.694479,0.0
France,0.702745,0.351969,-0.192017,0.712434,-0.927250,-6.0


# Test for exclusion restriction

In [41]:
import statsmodels.api as sm
from scipy.stats import pearsonr

# -------------------------------------------------
# 1.  Regress the Bartik instrument on the controls
# -------------------------------------------------
controls = [
    'distance_to_kyiv_km',      # proximity proxy
    'gdppc',                    # GDP per capita
    'energy_subsidies_increase',
    'unemployment_increase'
    # add other controls you want to partial-out here
]

X = dataset[controls]
X = sm.add_constant(X)          # adds an intercept
y = dataset['bartik_iv']        # the instrument

first_stage = sm.OLS(y, X).fit()
print(first_stage.summary())    # optional: inspect coefficients

# -------------------------------------------------
# 2.  Save the residuals  z̃ₗ  (Bartik purge of controls)
# -------------------------------------------------
dataset['bartik_resid'] = first_stage.resid

# -------------------------------------------------
# 3.  Correlate  z̃ₗ  with GDPpc and Proximity
# -------------------------------------------------
# (a) Pearson correlations
corr_gdppc,  pval_gdppc  = pearsonr(dataset['bartik_resid'], dataset['gdppc'])
corr_prox,   pval_prox   = pearsonr(dataset['bartik_resid'], dataset['distance_to_kyiv_km'])

print(f"Corr(resid, GDPpc)      = {corr_gdppc:.3f}  (p = {pval_gdppc:.3f})")
print(f"Corr(resid, Proximity)  = {corr_prox:.3f}   (p = {pval_prox:.3f})")

                            OLS Regression Results                            
Dep. Variable:              bartik_iv   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     3.669
Date:                Fri, 06 Jun 2025   Prob (F-statistic):             0.0235
Time:                        12:15:59   Log-Likelihood:                -25.267
No. Observations:                  23   AIC:                             60.53
Df Residuals:                      18   BIC:                             66.21
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [42]:
import statsmodels.api as sm

# --------------------------------------------------
# 1.  Bivariate check:   approval_change ~ GDPpc
# --------------------------------------------------
X1 = sm.add_constant(dataset['gdppc'])      # just GDP pc and an intercept
y  = dataset['approval_change']

biv = sm.OLS(y, X1).fit(cov_type='HC1')     # HC1 = robust SEs
print("=== Bivariate regression ===")
print(biv.summary())

# --------------------------------------------------
# 2.  Multivariate check:  approval_change ~ GDPpc + other controls
# --------------------------------------------------
controls = [
    'distance_to_kyiv_km',
    'energy_subsidies_increase',
    'unemployment_increase'
]
X2 = sm.add_constant(dataset[['gdppc'] + controls])

multi = sm.OLS(y, X2).fit(cov_type='HC1')
print("\n=== Multivariate regression ===")
print(multi.summary())

# --------------------------------------------------
# 3.  Easy-to-read takeaway
# --------------------------------------------------
coef  = multi.params['gdppc']
pval  = multi.pvalues['gdppc']
effect = "does NOT" if pval > 0.10 else "does"

print(f"\nPlain-English check: GDP per capita {effect} meaningfully predict "
      f"approval changes (coef = {coef:.3f}, p = {pval:.3f}).")


=== Bivariate regression ===
                            OLS Regression Results                            
Dep. Variable:        approval_change   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                     1.278
Date:                Fri, 06 Jun 2025   Prob (F-statistic):              0.271
Time:                        12:15:59   Log-Likelihood:                -66.329
No. Observations:                  23   AIC:                             136.7
Df Residuals:                      21   BIC:                             138.9
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.1304  

Add GDP pc in first stage and check if it is still significant

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Load inflation data
wb = pd.read_csv('../Design/data/world_inflation.csv', skiprows=4)

# Load GDP per capita (already normalized in your previous dataset)
gdppc = pd.read_csv('data/controls/GDPperCap.csv', index_col=0)
gdppc = gdppc[['2022']].rename(columns={'2022': 'gdppc'})

# Merge GDP with Bartik
bartik = bartik.reset_index().merge(gdppc, left_on='country', right_index=True, how='left')

# Function with GDP_pc control
def compute_inflation_shift_with_gdp(year1, year2, color, label, y_offset):
    wb_diff = wb[['Country Name', year1, year2]].rename(
        columns={year1: f'inflation_{year1}', year2: f'inflation_{year2}'}
    )
    wb_diff = wb_diff.dropna()
    wb_diff[f'inflation_{year1}'] = pd.to_numeric(wb_diff[f'inflation_{year1}'], errors='coerce')
    wb_diff[f'inflation_{year2}'] = pd.to_numeric(wb_diff[f'inflation_{year2}'], errors='coerce')
    wb_diff['inflation_change'] = wb_diff[f'inflation_{year2}'] - wb_diff[f'inflation_{year1}']

    merged = bartik.merge(wb_diff[['Country Name', 'inflation_change']],
                          left_on='country', right_on='Country Name', how='left')
    merged = merged[merged['country'].isin(europe_or_nearby)]
    merged = merged.dropna(subset=['inflation_change'])
    merged = merged[merged['inflation_change'].abs() <= 30]

    x = merged['bartik_iv'].values
    y = merged['inflation_change'].values
    gdp = merged['gdppc'].values
    names = merged['country'].values

    # OLS with GDP control
    X_ols = sm.add_constant(np.column_stack([x, gdp]))
    model = sm.OLS(y, X_ols).fit()
    y_pred = model.predict(X_ols)

    return model, names

# 2022 in blue (current first-stage)
model_2022, names_2022 = compute_inflation_shift_with_gdp('2021', '2022', 'blue', '2022 shift', y_offset=0.6)

# 2021 in red (placebo first-stage)
model_2021, names_2021 = compute_inflation_shift_with_gdp('2020', '2021', 'red', '2021 shift', y_offset=0.2)


# Print first-stage F for Bartik conditional on GDP_pc
print(f"2022 first-stage F (conditional on GDP): {model_2022.tvalues[1]**2:.2f}")
print(f"2021 placebo-stage F (conditional on GDP): {model_2021.tvalues[1]**2:.2f}")


2022 first-stage F (conditional on GDP): 12.70
2021 placebo-stage F (conditional on GDP): 1.44
